参数管理

首先关注具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8,1))

In [3]:
X = torch.rand(size=(2,4))
net(X)

tensor([[-0.4454],
        [-0.3780]], grad_fn=<AddmmBackward0>)

In [4]:
print(X)

tensor([[0.0217, 0.5857, 0.9076, 0.1539],
        [0.6070, 0.2604, 0.5620, 0.1450]])


参数访问

In [5]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2018, -0.1675, -0.2158, -0.2928, -0.1480, -0.3345, -0.3189,  0.1393]])), ('bias', tensor([-0.3052]))])


目标参数

In [6]:
print("type(net[2].bias):",type(net[2].bias),"\n")
print("net[2].bias:","\n",net[2].bias,"\n")
print("net[2].bias.data:","\n",net[2].bias.data,"\n")

type(net[2].bias): <class 'torch.nn.parameter.Parameter'> 

net[2].bias: 
 Parameter containing:
tensor([-0.3052], requires_grad=True) 

net[2].bias.data: 
 tensor([-0.3052]) 



In [7]:
net[2].weight.grad == None

True

一次性访问所有参数

In [9]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


当然可以。

首先，让我们回顾一下您给出的代码。您定义了一个简单的神经网络`net`，该网络由两个线性层（`nn.Linear`）和一个ReLU激活函数（`nn.ReLU`）组成。第一个线性层从4个输入特征映射到8个输出特征，第二个线性层从8个输入特征映射到1个输出特征。

接着，您创建了一个2x4的随机输入张量`X`，并通过`net(X)`将其传递给网络。

最后，您使用了一个列表推导式来打印网络第一个线性层的参数名称和形状。

现在，我们来详细解释`print(*[(name, param.shape) for name, param in net[0].named_parameters()])`这行代码：

1. **net[0].named_parameters()**：


	* `net[0]`：这是获取`net`中的第一个模块，即第一个线性层。
	* `named_parameters()`：这是一个方法，用于返回模块中所有参数的迭代器，其中每个参数都与其名称相关联。
2. **for name, param in net[0].named_parameters()**：


	* 这是一个for循环，用于遍历`net[0]`（即第一个线性层）的所有参数。
	* 在每次迭代中，`name`是参数的名称（例如`weight`或`bias`），`param`是参数的实际值。
3. **[(name, param.shape) for name, param in net[0].named_parameters()]**：


	* 这是一个列表推导式，用于创建一个新的列表。
	* 对于`net[0]`中的每个参数，它都会创建一个元组`(name, param.shape)`，其中`name`是参数的名称，`param.shape`是参数的形状。
4. **print(*...)**：


	* `*`操作符是Python中的解包操作符。在这里，它用于将列表推导式生成的列表解包为单独的参数，以便`print`函数可以打印它们。
	* 因此，如果您有一个元组列表`[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]`，则`print(*...)`将等同于`print('weight', torch.Size([8, 4]), 'bias', torch.Size([8]))`。

综上所述，该行代码的目的是打印网络第一个线性层的所有参数的名称和形状。对于给定的网络结构，输出将是：


```python
weight torch.Size([8, 4]) bias torch.Size([8])
```
这表示第一个线性层有一个形状为[8, 4]的权重矩阵和一个形状为[8]的偏置向量。

In [10]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [11]:
net.state_dict()['2.bias'].data

tensor([-0.3052])

从嵌套块收集参数

In [18]:
def block1():
    return nn.Sequential(nn.Linear(4,8), 
                         nn.ReLU(), 
                         nn.Linear(8,4),
                         nn.ReLU()
                        )


In [19]:
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net


In [20]:
rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[0.1036],
        [0.1036]], grad_fn=<AddmmBackward0>)

In [22]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


内置初始化

In [24]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        

In [25]:
net.apply(init_normal)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [26]:
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0031, -0.0035,  0.0027,  0.0103]), tensor(0.))

这些代码涉及PyTorch的神经网络初始化。我会为你详细解释每一部分。

1. **定义初始化函数 `init_normal`**:


```python
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
```
这个函数的目的是为神经网络中的线性层（`nn.Linear`）设置特定的权重和偏置初始化。

* `if type(m) == nn.Linear:`：这一行检查传入的模块`m`是否是线性层。
* `nn.init.normal_(m.weight, mean=0, std=0.01)`：这一行将线性层的权重初始化为均值为0、标准差为0.01的正态分布。注意，`_`后缀表示这个操作会直接在`m.weight`上进行原地修改，而不是返回一个新的张量。
* `nn.init.zeros_(m.bias)`：这一行将线性层的偏置初始化为0。同样，`_`后缀表示这是一个原地操作。
2. **应用初始化函数**:


```python
net.apply(init_normal)
```
net.apply()`是一个方法，它会遍历网络中的每一个模块，并对每一个模块应用`init_normal`函数。这意味着网络中的所有线性层都将使用上述定义的初始化方法。

3. **检查初始化后的权重和偏置**:


```python
net[0].weight.data[0], net[0].bias.data[0]
```
这一行代码会返回网络第一个线性层的第一个权重的值和第一个偏置的值。

* `net[0]`：这表示网络的第一个模块，即第一个线性层。
* `net[0].weight.data[0]`：这表示第一个线性层的权重的第一个元素。
* `net[0].bias.data[0]`：这表示第一个线性层的偏置的第一个元素。

综上所述，这些代码的目的是为网络中的所有线性层设置特定的权重和偏置初始化，并随后检查第一个线性层的初始化后的权重和偏置值。

In [28]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

In [29]:
net.apply(init_constant)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [30]:
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

当然可以。这段代码的目的是为了初始化神经网络中的某些层的权重和偏置。这里我们详细解释每一部分：

1. **定义初始化函数**：


```python
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
```
这个函数`init_constant`接收一个参数`m`，它代表网络中的一个模块或层。然后，该函数检查这个模块是否是线性层（即全连接层）。如果是，它将权重（`weight`）初始化为常数1，并将偏置（`bias`）初始化为0。

* `nn.init.constant_(m.weight, 1)`：这行代码将`m`的权重初始化为常数1。注意，`_`后缀表示这是一个就地操作，它会直接修改输入参数`m.weight`的值，而不是返回一个新的张量。
* `nn.init.zeros_(m.bias)`：这行代码将`m`的偏置初始化为0。同样，这也是一个就地操作。
2. **应用初始化函数**：


```python
net.apply(init_constant)
```
这行代码将整个网络`net`的每一个模块或层都传递给`init_constant`函数进行初始化。`net.apply()`是一个方便的方法，它遍历网络中的每一个模块，并将每个模块作为参数传递给给定的函数。

3. **检查初始化结果**：


```python
net[0].weight.data[0], net[0].bias.data[0]
```
这行代码检查网络`net`的第一个模块（通常是第一个层）的权重和偏置的第一个元素的值。由于我们使用了`init_constant`函数进行初始化，所以权重的第一个元素应该是1，偏置的第一个元素应该是0。

总结：这段代码的目的是将神经网络`net`中的所有线性层的权重初始化为常数1，偏置初始化为0，并检查第一个线性层的权重和偏置的第一个元素的值。

为什么权重不建议设置为常识

权重是机器学习模型中的一个关键参数，它决定了模型在训练过程中对不同数据点的重视程度。权重设置得是否合适，直接影响着模型的训练效果和泛化能力。在机器学习领域，通常不建议将权重设置为常数（如1或0）或基于某种固定的“常识”来设置，原因有以下几点：

1. **数据不平衡**：在实际应用中，数据集通常是不平衡的，即某些类别的样本数量可能远多于其他类别。如果简单地使用常识来设置权重，可能会导致模型偏向于数量较多的类别，从而忽略或错误地分类数量较少的类别。
2. **模型泛化能力**：机器学习模型的目标是从训练数据中学习到一般性的规律，从而能够泛化到未见过的数据上。如果权重设置过于简单或固定，模型可能无法充分学习到数据的复杂性，导致泛化能力下降。
3. **优化难度**：在模型训练过程中，权重通常是通过优化算法（如梯度下降）来自动调整的。如果初始权重设置不合理，可能会导致优化过程变得困难或陷入局部最优解。
4. **可解释性**：虽然基于常识设置权重可能使得模型更易于解释，但这种做法往往忽略了数据的复杂性。在很多情况下，数据的真实分布和模式可能比基于常识的假设更为复杂。

因此，为了获得更好的训练效果和泛化能力，通常建议让模型自动学习权重。这可以通过使用适当的优化算法和损失函数来实现，例如使用交叉熵损失函数来训练分类模型，并使用梯度下降等优化算法来更新权重。同时，还可以通过一些技巧来平衡不同类别的样本，例如使用过采样或欠采样方法来处理不平衡数据集。

对某些块应用不同的初始化方法

In [36]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

In [37]:
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

这段代码定义了一个名为 `xavier` 的函数，该函数用于初始化 PyTorch 神经网络中的线性层（全连接层）的权重。这个函数使用了 Xavier（或Glorot）均匀初始化方法，这是一种常用的权重初始化策略。下面是对代码的详细解释：

1. `def xavier(m):`：定义了一个名为 `xavier` 的函数，该函数接受一个参数 `m`，通常代表神经网络中的一个模块（比如一个线性层）。
2. `if type(m) == nn.Linear:`：这是一个条件语句，用于检查传入的模块 `m` 是否是线性层（`nn.Linear`）。只有当 `m` 是线性层时，接下来的初始化代码才会执行。
3. `nn.init.xavier_uniform_(m.weight)`：这一行是初始化代码的核心。`nn.init.xavier_uniform_` 是 PyTorch 中的一个函数，用于对张量进行 Xavier 均匀初始化。这里的下划线 `_` 表示这个函数是原地操作（in-place operation），即它会直接修改传入的张量，而不是返回一个新的张量。


	* `xavier_uniform_` 函数接受一个张量作为参数，并将其初始化为从均匀分布中抽取的值。这个均匀分布的区间是 `[-a, a]`，其中 `a` 是根据张量的形状计算出来的一个常数，目的是使初始化后的权重具有合适的尺度。
	* Xavier 均匀初始化是一种旨在改善神经网络训练效果的权重初始化策略。它的基本思想是使每一层神经元的输入和输出的方差尽可能相等，从而避免在训练过程中出现梯度消失或梯度爆炸的问题。

总体来说，这段代码定义了一个函数，用于将神经网络中的线性层的权重初始化为 Xavier 均匀分布的值。这是一种常用的权重初始化方法，有助于改善神经网络的训练稳定性和性能。

In [38]:
net[0].apply(xavier)

Linear(in_features=4, out_features=8, bias=True)

In [39]:
net[2].apply(init_42)

Linear(in_features=8, out_features=1, bias=True)

In [41]:
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.6285, -0.3778, -0.5230, -0.3246])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [42]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "init",
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

这段代码定义了一个名为 `my_init` 的函数，该函数用于初始化 PyTorch 神经网络中的模块，特别是线性层 (`nn.Linear`)。这个函数使用了自定义的权重初始化方法，并且打印了初始化过程中的一些信息。下面是对代码的详细解释：

1. `def my_init(m):`：定义了一个名为 `my_init` 的函数，该函数接受一个参数 `m`，通常代表神经网络中的一个模块。

2. `if type(m) == nn.Linear:`：这是一个条件语句，用于检查传入的模块 `m` 是否是线性层（`nn.Linear`）。只有当 `m` 是线性层时，接下来的初始化代码才会执行。

3. `print("init", *[(name, param.shape) for name, param in m.named_parameters()][0])`：这行代码打印一条消息，表明正在对某个模块进行初始化，并显示该模块的第一个参数的名称和形状。`m.named_parameters()` 是一个生成器，它返回模块中所有参数的名称和值。列表推导式 `[(name, param.shape) for name, param in m.named_parameters()]` 用于提取所有参数的名称和形状，并创建一个列表。由于我们知道线性层通常只有两个参数（`weight` 和 `bias`），所以列表推导式的结果是一个包含两个元组的列表。通过索引 `[0]`，我们获取第一个元组（即权重的名称和形状），并使用 `*` 解包操作符将其解包为单独的参数，以便 `print` 函数可以打印它们。

4. `nn.init.uniform_(m.weight, -10, 10)`：这行代码使用均匀分布初始化线性层的权重。权重将被设置为在 `-10` 和 `10` 之间的随机值。下划线 `_` 表示这是一个原地操作，它会直接修改 `m.weight` 的值。

5. `m.weight.data *= m.weight.data.abs() >= 5`：这行代码是一个逐元素的操作，用于进一步修改权重的值。它首先计算 `m.weight.data.abs() >= 5`，这会返回一个与权重形状相同的布尔张量，其中每个元素表示对应权重的绝对值是否大于或等于 5。然后，这个布尔张量被用作掩码，与原始权重进行逐元素的乘法操作。这意味着，如果权重的绝对值小于 5，它的值将被设置为 0；如果权重的绝对值大于或等于 5，它的值将保持不变。这种操作可以看作是一种硬阈值函数，用于将权重设置为 0 或保持其原始值（如果它足够大）。

综上所述，`my_init` 函数是一个自定义的初始化函数，用于初始化 PyTorch 神经网络中的线性层。它使用均匀分布初始化权重，并通过一个硬阈值函数将绝对值较小的权重设置为 0。在初始化过程中，它还打印了权重的名称和形状。

In [43]:
net.apply(my_init)

init weight torch.Size([8, 4])
init weight torch.Size([1, 8])


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [44]:
net[0].weight[:2]

tensor([[-5.5063,  0.0000, -0.0000,  9.2689],
        [-5.0918, -6.0503, -0.0000,  6.7154]], grad_fn=<SliceBackward0>)

In [47]:
net[0].weight.data[:] += 1
print(net[0].weight)

Parameter containing:
tensor([[-2.5063,  3.0000,  3.0000, 12.2689],
        [-2.0918, -3.0503,  3.0000,  9.7154],
        [ 3.0000, 11.4161, -4.5418, 12.3792],
        [12.4252,  3.0000, -2.4871, -5.7381],
        [11.8848,  3.0000,  3.0000,  3.0000],
        [ 3.0000, -2.2451,  3.0000, 12.9304],
        [-3.6574,  3.0000,  3.0000,  3.0000],
        [ 9.5005, -2.3450,  3.0000,  3.0000]], requires_grad=True)


In [48]:
net[0].weight.data[0,0] = 42
print(net[0].weight)

Parameter containing:
tensor([[42.0000,  3.0000,  3.0000, 12.2689],
        [-2.0918, -3.0503,  3.0000,  9.7154],
        [ 3.0000, 11.4161, -4.5418, 12.3792],
        [12.4252,  3.0000, -2.4871, -5.7381],
        [11.8848,  3.0000,  3.0000,  3.0000],
        [ 3.0000, -2.2451,  3.0000, 12.9304],
        [-3.6574,  3.0000,  3.0000,  3.0000],
        [ 9.5005, -2.3450,  3.0000,  3.0000]], requires_grad=True)


In [49]:
net[0].weight.data[0]

tensor([42.0000,  3.0000,  3.0000, 12.2689])

参数绑定

In [50]:
shared = nn.Linear(8, 8)

In [53]:
net = nn.Sequential(nn.Linear(4, 8),
                    nn.ReLU(),
                    shared,
                    nn.ReLU(),
                    shared,
                    nn.ReLU(),
                    nn.Linear(8,1)
                   )

In [54]:
net(X)

tensor([[-0.0252],
        [-0.0444]], grad_fn=<AddmmBackward0>)

In [55]:
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [57]:
net[2].weight.data[0, 0] = 100

In [58]:
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
